# Mulitple Linear Regression - 01

## Load the data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('data/full.csv')

In [4]:
data['GW_MEAS_DATE'] = pd.to_numeric(pd.to_datetime(data['GW_MEAS_DATE']))

In [5]:
import sklearn.linear_model as lm
import sklearn.model_selection as ms
import sklearn.metrics as metrics

In [6]:
from collections import Counter

# Create a Counter object for the WID column.
counter = Counter(data['WID'])

# Create a list of WID values that have 100 or more depth samples.
wids = []
for i in counter.items():
    if i[1] >= 100:
        wids.append(i[0])

In [22]:
from scipy import stats
from itertools import combinations

# Load linear regression model.
model = lm.LinearRegression()

cols = ['GW_MEAS_DATE', 'PRCP', 'TMAX', 'TMIN', 'ELEVATION']

# Create blank dataframe for the evaluations.
blank = pd.DataFrame(columns=['WID','R2','ADJ_R2','MAE','MSE','RMSE'])

f_res = pd.DataFrame(columns=['PARA','R2','ADJ_R2','MAE','MSE','RMSE'])
f_res_count = 0

t_count = 31

for n in range(1, 6):
        combos = list(combinations(cols, n))
        for para in combos:
                parameters = list(para)
                para_id = "['{}']".format("', '".join(parameters))
                
                # Copy the blank dataframe to the results dataframe.
                res = blank.copy()

                # Set count for results dataframe.
                res_count = 0

                # Loop through the WID values. These have 100 or more DEPTH samples.
                for j in wids:
                        try:
                                # Create temp dataframe from the blank dataframe to store specific well evaluation results.
                                tdf = blank.copy()

                                # Set count for temp dataframe.
                                tdf_count = 0

                                # Create subset of the data dataframe for the specific well.
                                df = data[data['WID'] == j].copy()

                                # Sort Date values
                                df.sort_values(by='GW_MEAS_DATE', inplace=True)

                                df.reset_index(drop=True, inplace=True)

                                # Loop through model processing 30 times to get an average of the results.
                                for n in range(30):

                                        x_train, x_test, y_train, y_test = ms.train_test_split(
                                                df[parameters], df['DEPTH'], test_size=0.2
                                                )

                                        #x_train = x_train.to_numpy().reshape(-1, 1)
                                        #x_test = x_test.to_numpy().reshape(-1, 1)
                                        y_train = y_train.to_numpy()
                                        y_test = y_test.to_numpy()

                                        model.fit(x_train, y_train)

                                        y_pred = model.predict(x_test)

                                        r2 = metrics.r2_score(y_test, y_pred)
                                        adj_r2 = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - x_train.shape[1] - 1)
                                        mae = metrics.mean_absolute_error(y_test, y_pred)
                                        mse = metrics.mean_squared_error(y_test, y_pred)
                                        rmse = np.sqrt(mse)

                                        tdf.loc[tdf_count] = [j, r2, adj_r2, mae, mse, rmse]
                                        tdf_count += 1

                                # Add the temp dataframe to the results dataframe.
                                tdf = tdf.mean(numeric_only=True).to_frame().T.reset_index().rename(columns={'index':'WID'})
                                tdf['WID'] = j
                                res.loc[res_count] = tdf.loc[0]
                                res_count += 1
                                
                        except:
                                print(f'{j} failed')
                                continue

                # Add the results dataframe to the final results dataframe.
                res_mean = res.mean(numeric_only=True).to_frame().T.reset_index().rename(columns={'index':'PARA'})
                res_mean['PARA'] = para_id
                f_res.loc[f_res_count] = res_mean.loc[0]
                f_res_count += 1
                pct = (f_res_count / t_count) * 100
                print(f'{pct:.2f}%')

['GW_MEAS_DATE']
3.23%
['PRCP']
6.45%
['TMAX']
9.68%
['TMIN']
12.90%
['ELEVATION']
16.13%
['GW_MEAS_DATE', 'PRCP']
19.35%
['GW_MEAS_DATE', 'TMAX']
22.58%
['GW_MEAS_DATE', 'TMIN']
25.81%
['GW_MEAS_DATE', 'ELEVATION']
29.03%
['PRCP', 'TMAX']
32.26%
['PRCP', 'TMIN']
35.48%
['PRCP', 'ELEVATION']
38.71%
['TMAX', 'TMIN']
41.94%
['TMAX', 'ELEVATION']
45.16%
['TMIN', 'ELEVATION']
48.39%
['GW_MEAS_DATE', 'PRCP', 'TMAX']
51.61%
['GW_MEAS_DATE', 'PRCP', 'TMIN']
54.84%
['GW_MEAS_DATE', 'PRCP', 'ELEVATION']
58.06%
['GW_MEAS_DATE', 'TMAX', 'TMIN']
61.29%
['GW_MEAS_DATE', 'TMAX', 'ELEVATION']
64.52%
['GW_MEAS_DATE', 'TMIN', 'ELEVATION']
67.74%
['PRCP', 'TMAX', 'TMIN']
70.97%
['PRCP', 'TMAX', 'ELEVATION']
74.19%
['PRCP', 'TMIN', 'ELEVATION']
77.42%
['TMAX', 'TMIN', 'ELEVATION']
80.65%
['GW_MEAS_DATE', 'PRCP', 'TMAX', 'TMIN']
83.87%
['GW_MEAS_DATE', 'PRCP', 'TMAX', 'ELEVATION']
87.10%
['GW_MEAS_DATE', 'PRCP', 'TMIN', 'ELEVATION']
90.32%
['GW_MEAS_DATE', 'TMAX', 'TMIN', 'ELEVATION']
93.55%
['PRCP', 'TMA

In [89]:
f_res = f_res.sort_values(by='ADJ_R2', ascending=False).reset_index(drop=True)

In [90]:
f_res

,PARA,R2,ADJ_R2,MAE,MSE,RMSE,P_NUM
0,['GW_MEAS_DATE'],0.283922,0.263135,3.173545,29.712652,4.052565,1
1,"['GW_MEAS_DATE', 'TMIN']",0.298042,0.254980,3.135917,29.669274,4.028991,2
2,"['GW_MEAS_DATE', 'TMAX']",0.281759,0.237879,3.155987,29.744086,4.053046,2
3,"['GW_MEAS_DATE', 'PRCP']",0.277729,0.234235,3.176623,29.712233,4.070395,2
4,"['GW_MEAS_DATE', 'ELEVATION']",0.274277,0.230580,3.181042,29.747080,4.056204,2
5,"['GW_MEAS_DATE', 'TMAX', 'TMIN']",0.296633,0.229709,3.141175,29.786356,4.031478,3
6,"['GW_MEAS_DATE', 'PRCP', 'TMIN']",0.287919,0.220230,3.141171,29.675962,4.034753,3
7,"['GW_MEAS_DATE', 'TMAX', 'ELEVATION']",0.284592,0.216740,3.155838,30.034458,4.061731,3
8,"['GW_MEAS_DATE', 'TMIN', 'ELEVATION']",0.282453,0.213665,3.152395,29.893698,4.047929,3
9,"['GW_MEAS_DATE', 'PRCP', 'TMAX']",0.278091,0.209898,3.154258,29.625041,4.060153,3


In [93]:
print(f_res[['PARA', 'ADJ_R2']].head())
print(f_res[['PARA', 'ADJ_R2']].tail())

                            PARA    ADJ_R2
0               ['GW_MEAS_DATE']  0.263135
1       ['GW_MEAS_DATE', 'TMIN']  0.254980
2       ['GW_MEAS_DATE', 'TMAX']  0.237879
3       ['GW_MEAS_DATE', 'PRCP']  0.234235
4  ['GW_MEAS_DATE', 'ELEVATION']  0.230580
                                     PARA    ADJ_R2
26                       ['PRCP', 'TMAX'] -0.205589
27          ['PRCP', 'TMAX', 'ELEVATION'] -0.222742
28          ['PRCP', 'TMIN', 'ELEVATION'] -0.233207
29               ['PRCP', 'TMAX', 'TMIN'] -0.268674
30  ['PRCP', 'TMAX', 'TMIN', 'ELEVATION'] -0.386019


In [95]:
plt.figure(figsize=(12,10))
plt.bar(f_res['PARA'], f_res['ADJ_R2'])
plt.xlabel('Parameters')
plt.ylabel('ADJ_R2')
plt.grid(True)
plt.xticks(rotation=90)
plt.show()

In [52]:
def p_num(row):
    para = row['PARA']
    para.strip('[]')
    para.replace("'", "")
    para = para.split(',')
    
    para = len(para)
    return para


In [53]:
f_res['P_NUM'] = f_res.apply(p_num, axis=1)

In [81]:
plt.scatter(f_res['P_NUM'], f_res['ADJ_R2'])
plt.show()